<a href="https://colab.research.google.com/github/fairml-research/TEST/blob/main/ADULT_TRUE_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
np.random.seed(7)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white", palette="muted", color_codes=True, context="talk")
from IPython import display
%matplotlib inline
import sys 
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

import keras as ke
import keras.backend as K
from keras.layers import Input, Dense, Dropout
from keras.models import Model

def load_ICU_data(path):
    column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 
                    'marital_status', 'occupation', 'relationship', 'race', 'sex', 
                    'capital_gain', 'capital_loss', 'hours_per_week', 'country', 'target']
    input_data = (pd.read_csv(path, names=column_names, 
                              na_values="?", sep=r'\s*,\s*', engine='python'))
                  #.loc[lambda df: df['race'].isin(['White', 'Black'])])
    input_data = pd.concat([input_data, pd.read_csv('http://grarivincent.com/old/adult.test', names=column_names, na_values="?", sep=r'\s*,\s*', engine='python').loc[1:,:] ])

    input_data[['age','fnlwgt','education_num','capital_gain', 'capital_loss', 'hours_per_week']] = input_data[['age','fnlwgt','education_num','capital_gain', 'capital_loss', 'hours_per_week']].astype(int)
    
    # sensitive attributes; we identify 'race' and 'sex' as sensitive attributes
    sensitive_attribs = ['sex']
    Z = (input_data.loc[:, sensitive_attribs]
         .assign(sex=lambda df: (df['sex'] == 'Male').astype(int)))
    # targets; 1 when someone makes over 50k , otherwise 0
    y = input_data['target'].replace({'<=50K.': 0, '>50K.': 1, '>50K': 1, '<=50K': 0 })
    XNC = input_data.loc[:, ['race','age','country']]
    XNC = XNC.assign(race=lambda df: (df['race'] == 'White').astype(int))
    print(XNC.shape)
    XNC = (XNC
           .fillna('Unknown')
           .pipe(pd.get_dummies, columns = ['country'], drop_first=True))
    # features; note that the 'target' and sentive attribute columns are dropped
    X = (input_data
         .drop(columns=['target','race','age','sex','country'])
         .fillna('Unknown')
         .pipe(pd.get_dummies, columns = ['workclass', 'education', 
                    'marital_status', 'occupation', 'relationship']))
    G = pd.DataFrame({'G1': X['relationship_Wife'], 'G2': X['relationship_Husband']})
    X = X.drop(columns=['relationship_Wife', 'relationship_Husband'])

    print(f"features X: {X.shape[0]} samples, {X.shape[1]} attributes")
    print(f"features X_NC: {XNC.shape[0]} samples, {XNC.shape[1]} attributes")
    print(f"targets y: {y.shape[0]} samples")
    print(f"sensitives Z: {Z.shape[0]} samples, {Z.shape[1]} attributes")
    return X, XNC, y, Z, G

# load ICU data set
X, XNC, y, S, G = load_ICU_data('http://grarivincent.com/old/adult.data') 

(48841, 3)
features X: 48841 samples, 56 attributes
features X_NC: 48841 samples, 43 attributes
targets y: 48841 samples
sensitives Z: 48841 samples, 1 attributes


In [5]:
G

,G1,G2
0,0,0
1,0,1
2,0,0
3,0,1
4,1,0
...,...,...
16276,0,0
16277,0,0
16278,0,1
16279,0,0


In [6]:
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 
                'marital_status', 'occupation', 'relationship', 'race', 'sex', 
                'capital_gain', 'capital_loss', 'hours_per_week', 'country', 'target']
input_data = (pd.read_csv('http://grarivincent.com/old/adult.data', names=column_names, 
                          na_values="?", sep=r'\s*,\s*', engine='python'))
              #.loc[lambda df: df['race'].isin(['White', 'Black'])])

In [7]:
input_data

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [8]:
# split into train/test set
X_train, X_test, XNC_train, XNC_test, y_train, y_test, S_train, S_test, G_train, G_test = train_test_split(X, XNC, y, S, G.values, test_size=0.2, random_state=7)

# standardize the data
scaler = MinMaxScaler().fit(X_train)
scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)

scalerNC = MinMaxScaler().fit(XNC_train)
scale_dfNC = lambda df, scaler: pd.DataFrame(scalerNC.transform(df), columns=df.columns, index=df.index)

X_train, X_test = X_train.pipe(scale_df, scaler), X_test.pipe(scale_df, scaler)
XNC_train, XNC_test = XNC_train.pipe(scale_dfNC, scaler), XNC_test.pipe(scale_dfNC, scalerNC)
X_train, X_test, XNC_train, XNC_test, y_train, y_test = X_train.values, X_test.values, XNC_train.values, XNC_test.values, y_train.values, y_test.values

S_train, S_test =S_train.values, S_test.values

In [9]:
X_train.shape

(39072, 56)

In [10]:
XNC_train.shape

(39072, 43)

In [11]:
53+43+1

97

In [12]:
X_train

array([[0.0948343 , 0.8       , 0.14344143, ..., 0.        , 0.        ,
        1.        ],
       [0.09831035, 0.6       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.35573281, 0.8       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.11796308, 0.6       , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.0704316 , 0.8       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.14809064, 0.53333333, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [13]:


import torch
import torch.nn as nn
import torch.nn.functional as F
H = 15
H2 = 15
epochs_hgr=50


batch_size = 1024
num_epochs = 75
learning_rate = 0.001
batch_no = len(X_train) // batch_size
#X_train_inv = recon_X_batch_a.data.numpy()
#Y_train_inv = recon_Y_batch_a.data.numpy()

from sklearn.utils import shuffle
from torch.autograd import Variable
criterionMSE = nn.MSELoss()
import math
nb_a=200
from torch.nn import functional as F
def sigmoid(x):
  output = [1 / (1 + math.exp(-x)) for x in x]
  return output

class NN_b(nn.Module):
    def __init__(self):
        super(NN_b, self).__init__()
        self.fc1 = nn.Linear(99, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 8)
        self.fc4 = nn.Linear(8, 1)        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x

class NN_s(nn.Module):
    def __init__(self):
        super(NN_s, self).__init__()
        self.fc1_s = nn.Linear(1, 8)
        self.fc2_s = nn.Linear(8, 8)
        self.fc3_s = nn.Linear(8, 1)
        #self.fc4_s = nn.Linear(16, 1)

    def forward(self, x):
        x = self.fc1_s(x)
        x = torch.relu(x)
        x = self.fc2_s(x)
        x = torch.relu(x)
        x = self.fc3_s(x)
        #x = torch.relu(x)
        #x = self.fc4_s(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
m_NN_s = NN_s().to(device)
optimizer_s = torch.optim.Adam(m_NN_s.parameters(), lr=learning_rate)

model_Y_b = NN_b().cuda()
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer_Y = torch.optim.Adam(model_Y_b.parameters(), lr=learning_rate)
criterion_ADV =  nn.MSELoss()
#A0 = sensitive.mean() + sensitive.std()*Variable(torch.randn(data.shape[0]*nb_a, 1),requires_grad=False)
#recon_X_batch_a0, z_a0, recon_Y_batch_a0, mu_a0, logvar_a0 = model(data.repeat(nb_a,1).view(-1, 33),A0)
#recon_x, recon_xNC, S, U,  recon_y, mu, logvar = model(x_var, x_varNC, y_var,s_var)
from sklearn.metrics import accuracy_score
X_trainCUDA = Variable(torch.FloatTensor(X_train)).cuda()
XNC_trainCUDA = Variable(torch.FloatTensor(XNC_train)).cuda()
y_trainCUDA = Variable(torch.FloatTensor(np.expand_dims(y_train,axis = 1))).cuda()

X_testCUDA = Variable(torch.FloatTensor(X_test)).cuda()
XNC_testCUDA = Variable(torch.FloatTensor(XNC_test)).cuda()
y_testCUDA = Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).cuda()


G_trainCUDA = Variable(torch.FloatTensor(G_train)).cuda()
G_testCUDA =  Variable(torch.FloatTensor(G_test)).cuda()

table = [0,0,0,0,0,0,0,0]
for epoch in range(num_epochs):
    lambda_ADV =100
    #if epoch % 50 != 0:
        #x_train = Variable(torch.FloatTensor(X_train.values),requires_grad=True)
        #x_test = Variable(torch.FloatTensor(X_test.values),requires_grad=True)
        #print('Epoch {}'.format(epoch+1))
        
        #y_predX= model(x_train).data.numpy().T
        #y_predXinv= model(recon_X_batch_a0).data.numpy().T

        #accuracy = (y_train-np.squeeze(y_predX)).pow(2).sum()     
        #accuracycount = accuracy_score(Y_train_inv.squeeze(1), np.squeeze(y_predXinv))
        #print ("Train Accuracy:", accuracy)#,"Train Accuracy Count:", accuracycount, "CLP:", np.mean(logit_y_predX-logit_y_predXinv)**2)
    x_train, x_trainNC, ytrain, sens_train, gtrain = shuffle(X_train, XNC_train, np.expand_dims(y_train,axis = 1),  S_train, G_train)

    #A = s_var.mean() + s_var.std()*Variable(torch.randn(batch_size*nb_a, 1),requires_grad=False)
    # Mini batch learning
    for i in range(batch_no):
        print('Epoch: ', epoch, 'Batch:',i)
        start = i * batch_size
        end = start + batch_size

        x_var = Variable(torch.FloatTensor(x_train[start:end])).cuda()
        xNC_var = Variable(torch.FloatTensor(x_trainNC[start:end])).cuda()
        y_var = Variable(torch.FloatTensor(ytrain[start:end])).cuda()
        s_var = Variable(torch.FloatTensor(sens_train[start:end])).cuda()
        # Forward + Backward + Optimize
        Ypred_var0 = model_Y_b(torch.cat([xNC_var.view(-1, 43),x_var.view(-1, x_var.shape[1])],1)).detach()


        for l in range(50):
            optimizer_s.zero_grad()
            Spred_var = m_NN_s(torch.sigmoid(Ypred_var0))
            lossS = criterion(Spred_var, s_var)
            optimizer_s.step()

        optimizer_Y.zero_grad()
        Ypred_var = model_Y_b(torch.cat([xNC_var.view(-1, 43),x_var.view(-1, x_var.shape[1])],1))
        Spred_var = m_NN_s(torch.sigmoid(Ypred_var))
        lossS = criterion(Spred_var, s_var)
        lossY = criterion(Ypred_var, y_var)
        loss =  lossY
        if epoch >= 20:
            loss =  -lambda_ADV * lossS + lossY
        ret=lossS
        loss.backward()
        optimizer_Y.step()
        print(loss)

Ypred = torch.sigmoid(model_Y_b(torch.cat([XNC_trainCUDA.view(-1, 43),X_trainCUDA.view(-1, X_trainCUDA.shape[1])],1)))>0.5
Ho = (S_train==1)
Fe = (S_train==0)
odds=Ypred.float()[Ho].mean()/ Ypred.float()[Fe].mean()
PRULE= min(odds, 1/odds)


Ypredtest = torch.sigmoid(model_Y_b(torch.cat([XNC_testCUDA.view(-1, 43),X_testCUDA.view(-1, X_testCUDA.shape[1] )],1)))>0.5
Ho = (S_test==1)
Fe = (S_test==0)
odds=Ypredtest.float()[Ho].mean()/ Ypredtest.float()[Fe].mean()
PRULEtest= min(odds, 1/odds)

print('ACC Train',(Ypred==y_trainCUDA).sum()/X_train.shape[0], 'PRULE',PRULE, 'ACC Test',(Ypredtest==y_testCUDA).sum()/X_test.shape[0],'PRULEtest',PRULEtest)
table = np.vstack([table,[num_epochs, lambda_ADV, (Ypred.cpu().data.numpy()==y_trainCUDA.cpu().data.numpy()).sum()/X_train.shape[0]*100,PRULE.cpu().data.numpy(),ret.cpu().data.numpy(), 
                        (Ypredtest.cpu().data.numpy()==y_testCUDA.cpu().data.numpy()).sum()/X_test.shape[0]*100,PRULEtest.cpu().data.numpy(),ret.cpu().data.numpy()]])

np.savetxt('Adult_ZhangFairness', table[1:,])


Epoch:  0 Batch: 0
tensor(0.7167, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 1
tensor(0.7151, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 2
tensor(0.7124, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 3
tensor(0.7095, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 4
tensor(0.7074, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 5
tensor(0.7055, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 6
tensor(0.7048, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 7
tensor(0.7018, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 8
tensor(0.7015, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 Batch: 9
tensor(0.6999, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  0 

KeyboardInterrupt: ignored

In [16]:
table = [0,0,0,0,0,0,0,0]


In [17]:
table=pd.read_csv("/content/Adult_AdelFairness", sep=" ", header=None)

In [ ]:

for k in range(20):
    # split into train/test set
    X_train, X_test, XNC_train, XNC_test, y_train, y_test, S_train, S_test, G_train, G_test = train_test_split(X, XNC, y, S, G.values, test_size=0.2, random_state=7)

    # standardize the data
    scaler = MinMaxScaler().fit(X_train)
    scale_df = lambda df, scaler: pd.DataFrame(scaler.transform(df), columns=df.columns, index=df.index)

    scalerNC = MinMaxScaler().fit(XNC_train)
    scale_dfNC = lambda df, scaler: pd.DataFrame(scalerNC.transform(df), columns=df.columns, index=df.index)

    X_train, X_test = X_train.pipe(scale_df, scaler), X_test.pipe(scale_df, scaler)
    XNC_train, XNC_test = XNC_train.pipe(scale_dfNC, scaler), XNC_test.pipe(scale_dfNC, scalerNC)
    X_train, X_test, XNC_train, XNC_test, y_train, y_test = X_train.values, X_test.values, XNC_train.values, XNC_test.values, y_train.values, y_test.values

    S_train, S_test =S_train.values, S_test.values  
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    H = 15
    H2 = 15
    epochs_hgr=50

    batch_size = 512
    num_epochs = 25
    learning_rate = 0.001
    batch_no = len(X_train) // batch_size
    #X_train_inv = recon_X_batch_a.data.numpy()
    #Y_train_inv = recon_Y_batch_a.data.numpy()

    from sklearn.utils import shuffle
    from torch.autograd import Variable
    criterionMSE = nn.MSELoss()
    import math
    nb_a=200
    from torch.nn import functional as F
    def sigmoid(x):
      output = [1 / (1 + math.exp(-x)) for x in x]
      return output

    class NN_b(nn.Module):
        def __init__(self):
            super(NN_b, self).__init__()
            self.fc1_s = nn.Linear(16, 8)
            self.fc2_s = nn.Linear(8, 8)
            self.fc3_s = nn.Linear(8, 1)
            #self.fc4_s = nn.Linear(16, 1)

        def forward(self, x):
            x = self.fc1_s(x)
            x = torch.relu(x)
            x = self.fc2_s(x)
            x = torch.relu(x)
            x = self.fc3_s(x)
            #x = torch.relu(x)
            #x = self.fc4_s(x)
            return x



    class NN_z(nn.Module):
        def __init__(self):
            super(NN_z, self).__init__()
            self.fc1 = nn.Linear(99, 64)
            self.fc2 = nn.Linear(64, 32)
            self.fc3 = nn.Linear(32, 16)

        def forward(self, x):
            x = self.fc1(x)
            x = torch.relu(x)
            x = self.fc2(x)
            x = torch.relu(x)
            x = self.fc3(x)
            return x


    class NN_s(nn.Module):
        def __init__(self):
            super(NN_s, self).__init__()
            self.fc1_s = nn.Linear(16, 8)
            self.fc2_s = nn.Linear(8, 8)
            self.fc3_s = nn.Linear(8, 1)
            #self.fc4_s = nn.Linear(16, 1)

        def forward(self, x):
            x = self.fc1_s(x)
            x = torch.relu(x)
            x = self.fc2_s(x)
            x = torch.relu(x)
            x = self.fc3_s(x)
            #x = torch.relu(x)
            #x = self.fc4_s(x)
            return x


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    m_NN_s = NN_s().to(device)
    optimizer_s = torch.optim.Adam(m_NN_s.parameters(), lr=learning_rate)

    m_NN_z = NN_z().to(device)
    optimizer_z = torch.optim.Adam(m_NN_z.parameters(), lr=learning_rate)

    model_Y_b = NN_b().cuda()
    criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
    optimizer_Y = torch.optim.Adam(model_Y_b.parameters(), lr=learning_rate)
    criterion_ADV =  nn.MSELoss()
    #A0 = sensitive.mean() + sensitive.std()*Variable(torch.randn(data.shape[0]*nb_a, 1),requires_grad=False)
    #recon_X_batch_a0, z_a0, recon_Y_batch_a0, mu_a0, logvar_a0 = model(data.repeat(nb_a,1).view(-1, 33),A0)
    #recon_x, recon_xNC, S, U,  recon_y, mu, logvar = model(x_var, x_varNC, y_var,s_var)
    from sklearn.metrics import accuracy_score
    X_trainCUDA = Variable(torch.FloatTensor(X_train)).cuda()
    XNC_trainCUDA = Variable(torch.FloatTensor(XNC_train)).cuda()
    y_trainCUDA = Variable(torch.FloatTensor(np.expand_dims(y_train,axis = 1))).cuda()

    X_testCUDA = Variable(torch.FloatTensor(X_test)).cuda()
    XNC_testCUDA = Variable(torch.FloatTensor(XNC_test)).cuda()
    y_testCUDA = Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).cuda()


    G_trainCUDA = Variable(torch.FloatTensor(G_train)).cuda()
    G_testCUDA =  Variable(torch.FloatTensor(G_test)).cuda()
    for epoch in range(num_epochs):
        lambda_ADV =k+18
        #if epoch % 50 != 0:
            #x_train = Variable(torch.FloatTensor(X_train.values),requires_grad=True)
            #x_test = Variable(torch.FloatTensor(X_test.values),requires_grad=True)
            #print('Epoch {}'.format(epoch+1))
            
            #y_predX= model(x_train).data.numpy().T
            #y_predXinv= model(recon_X_batch_a0).data.numpy().T

            #accuracy = (y_train-np.squeeze(y_predX)).pow(2).sum()     
            #accuracycount = accuracy_score(Y_train_inv.squeeze(1), np.squeeze(y_predXinv))
            #print ("Train Accuracy:", accuracy)#,"Train Accuracy Count:", accuracycount, "CLP:", np.mean(logit_y_predX-logit_y_predXinv)**2)
        x_train, x_trainNC, ytrain, sens_train, gtrain = shuffle(X_train, XNC_train, np.expand_dims(y_train,axis = 1),  S_train, G_train)

        #A = s_var.mean() + s_var.std()*Variable(torch.randn(batch_size*nb_a, 1),requires_grad=False)
        # Mini batch learning
        for i in range(batch_no):
            print('Epoch: ', epoch, 'Batch:',i)
            start = i * batch_size
            end = start + batch_size

            x_var = Variable(torch.FloatTensor(x_train[start:end])).cuda()
            xNC_var = Variable(torch.FloatTensor(x_trainNC[start:end])).cuda()
            y_var = Variable(torch.FloatTensor(ytrain[start:end])).cuda()
            s_var = Variable(torch.FloatTensor(sens_train[start:end])).cuda()
            # Forward + Backward + Optimize
            Ypred_var0 = model_Y_b(torch.cat([xNC_var.view(-1, 43),x_var.view(-1, x_var.shape[1])],1)).detach()


            Z0 = m_NN_z(torch.cat([xNC_var.view(-1, 43),x_var.view(-1, x_var.shape[1])],1)).detach()

            for e in range(10):
                optimizer_Y.zero_grad()
                Ypred_var = model_Y_b(Z0)
                lossY = criterion(Ypred_var, y_var)
                lossY.backward()
                optimizer_Y.step()

            if epoch >= 10 :
                if (epoch ==50) & (i<=3):
                    for l in range(200):
                        optimizer_s.zero_grad()
                        Spred_var = m_NN_s(Z0)
                        lossS = criterion(Spred_var, s_var)
                        lossS.backward()
                        optimizer_s.step()
                for l in range(20):
                    optimizer_s.zero_grad()
                    Spred_var = m_NN_s(Z0)
                    lossS = criterion(Spred_var, s_var)
                    lossS.backward()
                    optimizer_s.step()

            optimizer_z.zero_grad()
            Ypred_var = model_Y_b(m_NN_z(torch.cat([xNC_var.view(-1, 43),x_var.view(-1, x_var.shape[1])],1)))
            Spred_var = m_NN_s(m_NN_z(torch.cat([xNC_var.view(-1, 43),x_var.view(-1, x_var.shape[1])],1)))
            lossS = criterion(Spred_var, s_var)
            lossY = criterion(Ypred_var, y_var)
            loss =  lossY
            if epoch >= 10:
                loss =  - lambda_ADV * lossS + lossY

            loss.backward()
            optimizer_z.step()
            print('Loss: ', loss, 'lossS:',lossS, 'lossY:',lossY)
            #if epoch % 10 == 0:

    Ypred = torch.sigmoid(model_Y_b(m_NN_z(torch.cat([XNC_trainCUDA.view(-1, 43),X_trainCUDA.view(-1, X_trainCUDA.shape[1])],1))))>0.5
    Ho = (S_train==1)
    Fe = (S_train==0)
    odds=Ypred.float()[Ho].mean()/ Ypred.float()[Fe].mean()
    PRULE= min(odds, 1/odds)

    Ypredtest = torch.sigmoid(model_Y_b(m_NN_z(torch.cat([XNC_testCUDA.view(-1, 43),X_testCUDA.view(-1, X_testCUDA.shape[1] )],1))))>0.5
    Ho = (S_test==1)
    Fe = (S_test==0)
    odds=Ypredtest.float()[Ho].mean()/ Ypredtest.float()[Fe].mean()
    PRULEtest= min(odds, 1/odds)

    print('ACC Train',(Ypred==y_trainCUDA).sum()/X_train.shape[0], 'PRULE',PRULE, 'ACC Test',(Ypredtest==y_testCUDA).sum()/X_test.shape[0],'PRULEtest',PRULEtest)
    table = np.vstack([table,[num_epochs, lambda_ADV, (Ypred.cpu().data.numpy()==y_trainCUDA.cpu().data.numpy()).sum()/X_train.shape[0]*100,PRULE.cpu().data.numpy(),lossS.cpu().data.numpy(), 
                            (Ypredtest.cpu().data.numpy()==y_testCUDA.cpu().data.numpy()).sum()/X_test.shape[0]*100,PRULEtest.cpu().data.numpy(),lossS.cpu().data.numpy()]])
    np.savetxt('/content/drive/MyDrive/Withoutdemo2/Adult_AdelFairness', table[1:,])


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Epoch:  3 Batch: 66
Loss:  tensor(0.3203, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) lossS: tensor(0.7227, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) lossY: tensor(0.3203, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  3 Batch: 67
Loss:  tensor(0.2934, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) lossS: tensor(0.7182, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) lossY: tensor(0.2934, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  3 Batch: 68
Loss:  tensor(0.3296, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) lossS: tensor(0.7162, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>) lossY: tensor(0.3296, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
Epoch:  3 Batch: 69
Loss:  tensor(0.3066, device='cuda:0', grad_fn=<BinaryCross

In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
H = 15
H2 = 15
epochs_hgr=50


table = [0,0,0,0,0,0,0,0,0,0,0,0]
batch_size = 512
num_epochs = 25
learning_rate = 0.0001
batch_no = len(X_train) // batch_size
#X_train_inv = recon_X_batch_a.data.numpy()
#Y_train_inv = recon_Y_batch_a.data.numpy()

from sklearn.utils import shuffle
from torch.autograd import Variable
criterionMSE = nn.MSELoss()
import math
nb_a=200
from torch.nn import functional as F
def sigmoid(x):
  output = [1 / (1 + math.exp(-x)) for x in x]
  return output

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x
class NN_a(nn.Module):
    def __init__(self):
        super(NN_a, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1]+1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x


model_Y_b = NN_b().cuda()
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer_Y = torch.optim.Adam(model_Y_b.parameters(), lr=0.001)

model_A = NN_a().cuda()
optimizer_A = torch.optim.Adam(model_A.parameters(), lr=0.001)



model_Y = NN().cuda()
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer_Y = torch.optim.Adam(model_Y.parameters(), lr=0.001)
criterion_ADV =  nn.MSELoss()
#A0 = sensitive.mean() + sensitive.std()*Variable(torch.randn(data.shape[0]*nb_a, 1),requires_grad=False)
#recon_X_batch_a0, z_a0, recon_Y_batch_a0, mu_a0, logvar_a0 = model(data.repeat(nb_a,1).view(-1, 33),A0)
#recon_x, recon_xNC, S, U,  recon_y, mu, logvar = model(x_var, x_varNC, y_var,s_var)
from sklearn.metrics import accuracy_score
X_trainCUDA = Variable(torch.FloatTensor(X_train)).cuda()
y_trainCUDA = Variable(torch.FloatTensor(np.expand_dims(y_train,axis = 1))).cuda()

X_testCUDA = Variable(torch.FloatTensor(X_test)).cuda()
y_testCUDA = Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).cuda()
for betamse0 in range(20):
    for o in range(10):
        for epoch in range(num_epochs):
            #if epoch % 50 != 0:
                #x_train = Variable(torch.FloatTensor(X_train.values),requires_grad=True)
                #x_test = Variable(torch.FloatTensor(X_test.values),requires_grad=True)
                #print('Epoch {}'.format(epoch+1))
                
                #y_predX= model(x_train).data.numpy().T
                #y_predXinv= model(recon_X_batch_a0).data.numpy().T

                #accuracy = (y_train-np.squeeze(y_predX)).pow(2).sum()     
                #accuracycount = accuracy_score(Y_train_inv.squeeze(1), np.squeeze(y_predXinv))
                #print ("Train Accuracy:", accuracy)#,"Train Accuracy Count:", accuracycount, "CLP:", np.mean(logit_y_predX-logit_y_predXinv)**2)
            x_train, ytrain, sens_train, gtrain = shuffle(X_train, np.expand_dims(y_train,axis = 1),  S_train, G_train)

            #A = s_var.mean() + s_var.std()*Variable(torch.randn(batch_size*nb_a, 1),requires_grad=False)
            # Mini batch learning
            for i in range(batch_no):
                print('Epoch: ', epoch, 'Batch:',i)
                start = i * batch_size
                end = start + batch_size

                x_var = Variable(torch.FloatTensor(x_train[start:end])).cuda()
                y_var = Variable(torch.FloatTensor(ytrain[start:end])).cuda()
                s_var = Variable(torch.FloatTensor(sens_train[start:end])).cuda()
                g_train = Variable(torch.FloatTensor(gtrain[start:end])).cuda()
                betamse =betamse0/10 #7   ## LAMBDA =0.7

                for l in range(1):
                  optimizer_A.zero_grad()
                  ypred_var0 = model_Y(torch.cat([x_var.view(-1, x_var.shape[1])],1)).detach()
                  fx = model_A(torch.cat([x_var.view(-1, x_var.shape[1]),y_var],1))
                  lambdab =  fx/torch.mean(fx)
                  lossY = -(y_var * torch._C._nn.log_sigmoid(ypred_var0) + (1 - y_var) * torch._C._nn.log_sigmoid(-ypred_var0))
                  lossA = -torch.mean(lossY +betamse*lambdab*lossY)
                  lossA.backward()
                  optimizer_A.step()


                optimizer_Y.zero_grad()
                fx = model_A(torch.cat([x_var.view(-1, x_var.shape[1]),y_var],1))
                lambdab =  fx/torch.mean(fx)
                ypred_var = model_Y(torch.cat([x_var.view(-1, x_var.shape[1])],1))
                #lossY =criterion(ypred_var, y_var)
                lossY = -(y_var * torch._C._nn.log_sigmoid(ypred_var) + (1 - y_var) * torch._C._nn.log_sigmoid(-ypred_var))
                loss = torch.mean(lossY +betamse*lambdab*lossY) # + lossX_inv -2*lossX#+ betamse*MSEcount
                loss.backward()
                optimizer_Y.step()

         
                Ypred =  model_Y(torch.cat([X_trainCUDA.view(-1, X_trainCUDA.shape[1])],1))>0.5
                Ho = (S_train==1)
                Fe = (S_train==0)
                odds=Ypred.float()[Ho].mean()/ Ypred.float()[Fe].mean()
                PRULE= min(odds, 1/odds)
                FP0=torch.abs(Ypred.float()[Ho & (y_trainCUDA.cpu().data.numpy()==0)].mean()-Ypred.float()[Fe & (y_trainCUDA.cpu().data.numpy()==0)].mean())
                FN1=torch.abs((1-Ypred.float()[Ho & (y_trainCUDA.cpu().data.numpy()==1)].mean())-(1-Ypred.float()[Fe & (y_trainCUDA.cpu().data.numpy()==1)].mean()))  

                Ypredtest = model_Y(torch.cat([X_testCUDA.view(-1, X_testCUDA.shape[1] )],1))>0.5      
                Ho = (S_test==1)
                Fe = (S_test==0)
                odds=Ypredtest.float()[Ho].mean()/ Ypredtest.float()[Fe].mean()
                PRULEtest= min(odds, 1/odds)
                FP0test=torch.abs(Ypredtest.float()[Ho & (y_testCUDA.cpu().data.numpy()==0)].mean()- Ypredtest.float()[Fe& (y_testCUDA.cpu().data.numpy()==0)].mean())
                FN1test=torch.abs((1-Ypredtest.float()[Ho & (y_testCUDA.cpu().data.numpy()==1)].mean())-(1-Ypredtest.float()[Fe & (y_testCUDA.cpu().data.numpy()==1)].mean()))  

                print('ACC Train',(Ypred==y_trainCUDA).sum()/X_train.shape[0], 'PRULE',PRULE, 'ACC Test',(Ypredtest==y_testCUDA).sum()/X_test.shape[0],'PRULEtest',PRULEtest)

        table = np.vstack([table,[ num_epochs, betamse,(Ypred.cpu().data.numpy()==y_trainCUDA.cpu().data.numpy()).sum()/X_train.shape[0]*100,PRULE.cpu().data.numpy(),ret.cpu().data.numpy(), FP0,FN1,
                                  (Ypredtest.cpu().data.numpy()==y_testCUDA.cpu().data.numpy()).sum()/X_test.shape[0]*100,PRULEtest.cpu().data.numpy(),ret.cpu().data.numpy(),FP0test,FN1test]])

    np.savetxt('Adult_AdelFairness', table[1:,])

In [ ]:
table.shape

(5, 8)

In [ ]:
Ypred = torch.sigmoid(model_Y_b(m_NN_z(torch.cat([XNC_trainCUDA.view(-1, 43),X_trainCUDA.view(-1, X_trainCUDA.shape[1])],1))))>0.5
Ho = (S_train==1)
Fe = (S_train==0)
odds=Ypred.float()[Ho].mean()/ Ypred.float()[Fe].mean()
PRULE= min(odds, 1/odds)

In [ ]:
Ypred.float()[Ho].mean()

tensor(0.0914, device='cuda:0')

In [ ]:
 ((torch.sigmoid(Spred_var)>0.5)==s_var).mean()

RuntimeError: ignored

In [ ]:
(torch.sigmoid(Spred_var)>0.5)==s_var

tensor([[ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [ True],
        [False],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [ True],
        [ True

In [ ]:
*Ypred_var.shape

torch.Size([256, 1])

In [ ]:
lamb =75
lambda_ADV =lamb
batch_size=128
bs = 1024
num_epochs = 500
learning_rate = 0.0001
batch_no = len(X_train) // bs
lamb

m_NN_y = NN_y().to(device)
m_NN_s = NN_s().to(device)

optimizer_y = torch.optim.Adam(m_NN_y.parameters(), lr=learning_rate)
optimizer_s = torch.optim.Adam(m_NN_s.parameters(), lr=learning_rate)

criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')

for epoch in range(num_epochs):
    x_train, ytrain, strain = shuffle(X_train,np.expand_dims(y_train,axis = 1),sensitive)
    # Mini batch learning
    epsilon=0.00000000000000001
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        x_var = Variable(torch.FloatTensor(x_train[start:end])).to(device)
        y_var = Variable(torch.FloatTensor(ytrain[start:end])).to(device)
        s_var = Variable(torch.FloatTensor(strain[start:end])).unsqueeze(1).to(device)
        # Forward + Backward + Optimize
        Ypred_var0 = m_NN_y(x_var).detach()
        for l in range(10):
            optimizer_s.zero_grad()
            Spred_var = m_NN_s(torch.sigmoid(Ypred_var0))
            lossS = criterion(Spred_var, s_var)
            optimizer_s.step()

        optimizer_y.zero_grad()
        Ypred_var = m_NN_y(x_var)
        Spred_var = m_NN_s(torch.sigmoid(Ypred_var))
        lossS = criterion(Spred_var, s_var)
        lossY = criterion(Ypred_var, y_var)
        loss =  lossY
        if epoch >= 200:
            loss = - lambda_ADV * lossS + lossY

        loss.backward()
        optimizer_y.step()
    if epoch % 5 == 0:
        y_pred2= torch.sigmoid(m_NN_y(torch.FloatTensor(X_train).to(device))).cpu().data.numpy()
        y_pred2t= torch.sigmoid(m_NN_y(torch.FloatTensor(X_test).to(device))).cpu().data.numpy()
        print('epoch', epoch, 'loss', loss.cpu().data.numpy(), 'lossS', lossS.cpu().data.numpy(), 'lossY', lossY.cpu().data.numpy(),'P-rule', p_rule(y_pred2,np.expand_dims(sensitive,1)),'ACC_train',accuracy_score(y_train, y_pred2>0.5),
              'P-ruletest', p_rule(y_pred2t,np.expand_dims(sensitivet,1)),'ACC_test',accuracy_score(y_test, y_pred2t>0.5))


NameError: ignored

In [ ]:
s_var.shape

torch.Size([512, 1])

In [ ]:
optimizer_Y.step()

In [ ]:
(Ypredtest.cpu().data.numpy()==y_testCUDA.cpu().data.numpy()).sum()/X_test.shape[0]*100

85.45398710205752

HGR



# Variational Autoencoder

In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
H = 15
H2 = 15
epochs_hgr=50


table = [0,0,0,0,0,0,0,0]
batch_size = 512
num_epochs = 25
learning_rate = 0.0001
batch_no = len(X_train) // batch_size
#X_train_inv = recon_X_batch_a.data.numpy()
#Y_train_inv = recon_Y_batch_a.data.numpy()

from sklearn.utils import shuffle
from torch.autograd import Variable
criterionMSE = nn.MSELoss()
import math
nb_a=200
from torch.nn import functional as F
def sigmoid(x):
  output = [1 / (1 + math.exp(-x)) for x in x]
  return output

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(99, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(16, 1)        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        return x

model_Y = NN().cuda()
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')
optimizer_Y = torch.optim.Adam(model_Y.parameters(), lr=0.001)
criterion_ADV =  nn.MSELoss()
#A0 = sensitive.mean() + sensitive.std()*Variable(torch.randn(data.shape[0]*nb_a, 1),requires_grad=False)
#recon_X_batch_a0, z_a0, recon_Y_batch_a0, mu_a0, logvar_a0 = model(data.repeat(nb_a,1).view(-1, 33),A0)
#recon_x, recon_xNC, S, U,  recon_y, mu, logvar = model(x_var, x_varNC, y_var,s_var)
from sklearn.metrics import accuracy_score
X_trainCUDA = Variable(torch.FloatTensor(X_train)).cuda()
XNC_trainCUDA = Variable(torch.FloatTensor(XNC_train)).cuda()
y_trainCUDA = Variable(torch.FloatTensor(np.expand_dims(y_train,axis = 1))).cuda()

X_testCUDA = Variable(torch.FloatTensor(X_test)).cuda()
XNC_testCUDA = Variable(torch.FloatTensor(XNC_test)).cuda()
y_testCUDA = Variable(torch.FloatTensor(np.expand_dims(y_test,axis = 1))).cuda()
for betamse0 in range(20):
    for o in range(10):
        for epoch in range(num_epochs):
            #if epoch % 50 != 0:
                #x_train = Variable(torch.FloatTensor(X_train.values),requires_grad=True)
                #x_test = Variable(torch.FloatTensor(X_test.values),requires_grad=True)
                #print('Epoch {}'.format(epoch+1))
                
                #y_predX= model(x_train).data.numpy().T
                #y_predXinv= model(recon_X_batch_a0).data.numpy().T

                #accuracy = (y_train-np.squeeze(y_predX)).pow(2).sum()     
                #accuracycount = accuracy_score(Y_train_inv.squeeze(1), np.squeeze(y_predXinv))
                #print ("Train Accuracy:", accuracy)#,"Train Accuracy Count:", accuracycount, "CLP:", np.mean(logit_y_predX-logit_y_predXinv)**2)
            x_train, x_trainNC, ytrain, sens_train, gtrain = shuffle(X_train, XNC_train, np.expand_dims(y_train,axis = 1),  S_train, G_train)

            #A = s_var.mean() + s_var.std()*Variable(torch.randn(batch_size*nb_a, 1),requires_grad=False)
            # Mini batch learning
            for i in range(batch_no):
                print('Epoch: ', epoch, 'Batch:',i)
                start = i * batch_size
                end = start + batch_size

                x_var = Variable(torch.FloatTensor(x_train[start:end])).cuda()
                xNC_var = Variable(torch.FloatTensor(x_trainNC[start:end])).cuda()
                y_var = Variable(torch.FloatTensor(ytrain[start:end])).cuda()
                s_var = Variable(torch.FloatTensor(sens_train[start:end])).cuda()
                g_train = Variable(torch.FloatTensor(gtrain[start:end])).cuda()

                fx = model_Y_b(torch.cat([xNC_var.view(-1, 43),x_var.view(-1, x_var.shape[1])],1)).detach()

                optimizer_Y.zero_grad()
                bx = model_Y(torch.cat([xNC_var.view(-1, 43),x_var.view(-1, x_var.shape[1])],1))
                ypred_var = fx + (bx.T*(g_train[:,0]+g_train[:,1])).T
                lossY =criterion(ypred_var, y_var)
                betamse =betamse0/10 #7   ## LAMBDA =0.7
                ret = torch.mean(ypred_var) -   torch.mean(ypred_var*g_train[:,0]) -   torch.mean(ypred_var*g_train[:,1])
                loss = lossY +betamse*ret # + lossX_inv -2*lossX#+ betamse*MSEcount
                loss.backward()
                optimizer_Y.step()

                
                fxtrain = model_Y_b(torch.cat([XNC_trainCUDA.view(-1, 43),X_trainCUDA.view(-1, x_var.shape[1])],1)).detach()
                bxtrain= model_Y(torch.cat([XNC_trainCUDA.view(-1, 43),X_trainCUDA.view(-1, X_trainCUDA.shape[1])],1))
                Ypred =  torch.sigmoid(fxtrain + (bxtrain.T*(G_trainCUDA[:,0]+G_trainCUDA[:,1])).T)>0.5

                Ho = (S_train==1)
                Fe = (S_train==0)
                odds=Ypred.float()[Ho].mean()/ Ypred.float()[Fe].mean()
                PRULE= min(odds, 1/odds)

                fxtest = model_Y_b(torch.cat([XNC_testCUDA.view(-1, 43),X_testCUDA.view(-1, x_var.shape[1])],1)).detach()
                bxtest = model_Y(torch.cat([XNC_testCUDA.view(-1, 43),X_testCUDA.view(-1, X_testCUDA.shape[1] )],1))
                Ypredtest =  torch.sigmoid(fxtest + (bxtest.T*(G_testCUDA[:,0]+G_testCUDA[:,1])).T)>0.5

                Ho = (S_test==1)
                Fe = (S_test==0)
                odds=Ypredtest.float()[Ho].mean()/ Ypredtest.float()[Fe].mean()
                PRULEtest= min(odds, 1/odds)
                print('ACC Train',(Ypred==y_trainCUDA).sum()/X_train.shape[0], 'PRULE',PRULE, 'ACC Test',(Ypredtest==y_testCUDA).sum()/X_test.shape[0],'PRULEtest',PRULEtest)
        table = np.vstack([table,[num_epochs, betamse, (Ypred.cpu().data.numpy()==y_trainCUDA.cpu().data.numpy()).sum()/X_train.shape[0]*100,PRULE.cpu().data.numpy(),ret.cpu().data.numpy(), 
                                (Ypredtest.cpu().data.numpy()==y_testCUDA.cpu().data.numpy()).sum()/X_test.shape[0]*100,PRULEtest.cpu().data.numpy(),ret.cpu().data.numpy()]])

    np.savetxt('Adult_ProxyFairness', table[1:,])


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Epoch:  7 Batch: 43
ACC Train tensor(0.8862, device='cuda:0') PRULE tensor(0.2940, device='cuda:0') ACC Test tensor(0.8467, device='cuda:0') PRULEtest tensor(0.2786, device='cuda:0')
Epoch:  7 Batch: 44
ACC Train tensor(0.8863, device='cuda:0') PRULE tensor(0.2948, device='cuda:0') ACC Test tensor(0.8471, device='cuda:0') PRULEtest tensor(0.2811, device='cuda:0')
Epoch:  7 Batch: 45
ACC Train tensor(0.8864, device='cuda:0') PRULE tensor(0.2946, device='cuda:0') ACC Test tensor(0.8472, device='cuda:0') PRULEtest tensor(0.2865, device='cuda:0')
Epoch:  7 Batch: 46
ACC Train tensor(0.8866, device='cuda:0') PRULE tensor(0.2945, device='cuda:0') ACC Test tensor(0.8470, device='cuda:0') PRULEtest tensor(0.2894, device='cuda:0')
Epoch:  7 Batch: 47
ACC Train tensor(0.8871, device='cuda:0') PRULE tensor(0.2956, device='cuda:0') ACC Test tensor(0.8475, device='cuda:0') PRULEtest tensor(0.2905, device='cuda:0')
Epoch: 

KeyboardInterrupt: ignored

In [ ]:
fxtrain.shape

torch.Size([39072, 1])

In [ ]:
y_trainCUDA.shape

torch.Size([39072, 1])

In [ ]:
g_train[:,1]

tensor([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
        0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
        0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
        0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 0., 

In [ ]:
(torch.sigmoid(fx)>0.5)==y_var

tensor([[ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True

In [ ]:
(g_train[:,0]+g_train[:,1])

tensor([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
        0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
        0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
        1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        1., 0., 1., 1., 1., 0., 0., 0., 

In [ ]:
g_train[:,1]

tensor([0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
        0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0.,
        1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1.,
        0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
        0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 0., 

In [ ]:
(bx.T*g_train[:,0]).T

tensor([[-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-3.5246e+14],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0.0000e+00],
        [-0

In [ ]:
ypred_var

tensor([[-1.9911e+00],
        [-4.3086e+14],
        [-3.3302e+14],
        [-4.2306e+14],
        [-3.6673e+00],
        [-5.0481e+00],
        [-5.1035e+00],
        [-7.1505e+00],
        [-3.5063e+14],
        [-3.7704e+14],
        [-4.3549e+14],
        [-3.5246e+14],
        [-6.1639e+00],
        [-6.4046e+00],
        [-5.5026e+00],
        [-4.6898e+00],
        [-4.4032e+14],
        [-3.4225e+14],
        [-5.3682e+00],
        [-4.5587e+14],
        [-1.7998e+00],
        [-3.3343e+14],
        [-3.7069e+14],
        [-4.4304e+14],
        [-5.9034e+00],
        [-5.8294e+00],
        [-3.0609e+14],
        [-4.1332e+14],
        [-7.5818e+00],
        [-3.4790e+14],
        [-2.5957e+00],
        [-3.8634e+14],
        [-5.2786e+00],
        [-4.2858e+14],
        [-5.1333e+00],
        [-2.4065e+00],
        [-3.3925e+14],
        [-3.4660e+14],
        [-7.4886e+00],
        [-3.4250e+14],
        [-3.2546e+14],
        [-4.3069e+14],
        [-3.4838e+14],
        [-3

In [ ]:
a=(bx.T*g_train[:,0]).T
a.shape

torch.Size([512, 1])

In [ ]:
fx.shape

torch.Size([512, 1])

In [ ]:
G_train.shape

(39072, 2)

In [ ]:
gtrain.shape

(39072, 2)